In [ ]:
% pip install numpy
% pip install opencv
% pip install PIL
% pip install tensorflow
% pip install matplotlib
% pip install pathlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import pathlib
import warnings

In [ ]:
import pathlib

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = pathlib.Path(data_dir).with_suffix('')

In [ ]:
local_dataset_path = "image recognition model\\flower_photos"
data_dir = pathlib.Path(local_dataset_path).with_suffix('').resolve()

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
roses = list(data_dir.glob('roses/*'))
plt.figure(figsize=(10, 5))
for i in range(2):
    plt.subplot(1, 2, i + 1)
    image = PIL.Image.open(str(roses[i]))
    plt.imshow(image)
    plt.title('Rose Image {}'.format(i + 1))
    plt.axis('off')
plt.show()

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
plt.figure(figsize=(10, 5))    
for i in range(2):    
    plt.subplot(1, 2, i + 1)    
    image = PIL.Image.open(str(tulips[i]))    
    plt.imshow(image)
    plt.title('Tulip Image {}'.format(i + 1)) 
    plt.axis('off')  
plt.show()

In [ ]:
batch_size = 64
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.8,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    verbose=0)

In [ ]:
validation_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

In [ ]:
print("min value for pixel ", np.min(first_image))
print("max value for pixel ", np.max(first_image))

In [ ]:
def preprocess_image(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    return img_array

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal",
                          input_shape=(img_height,
                                       img_width,
                                       3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

In [ ]:
plt.figure(figsize=(8, 8))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.title('Augmentation sample')
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:

checkpoint_path = "PATH\\training_checkpoint/cp.weights.h5"

checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                    save_weights_only=True,
                                    verbose=1)

In [ ]:
num_classes = len(class_names)

model = Sequential([
    data_augmentation,
        layers.Rescaling(1. / 255, input_shape=(img_height, img_width, 3)),
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, name="outputs")
    ])

In [ ]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
iterations = 50
history = model.fit(
      train_ds,
      validation_data=validation_ds,
      epochs=iterations,
      callbacks=[checkpoint_callback]
)

In [ ]:
model.save('my_model.keras')
model.load_weights(checkpoint_path)
try:
    model.load_weights(checkpoint_path)
    print("Weights loaded successfully!")
except FileNotFoundError:
    print("Checkpoint file not found. Please verify the file path.")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

iterations_range = range(iterations)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(iterations_range, acc, label='Training Accuracy')
plt.plot(iterations_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(iterations_range, loss, label='Training Loss')
plt.plot(iterations_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
image_path = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
image_path = tf.keras.utils.get_file('Red_sunflower', origin=image_path)

model = load_model(model_path)
preprocessed_image = preprocess_image(image_path)
predictions = model.predict(preprocessed_image)
predicted_class_index = np.argmax(predictions)
class_names = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
predicted_class = class_names[predicted_class_index]
confidence = np.max(predictions) * 100

print(f"The image most likely belongs to {predicted_class} with a confidence of {confidence:.2f}%.")
